# Perform pre-processing

This version uses pandas and requires the data to fit in memory. TODO: Rewrite in PySpark

For data I use:

    Spotify Playlists Dataset
    By Martin Pichl;  Eva Zangerle
    For more details see: zenodo.org/record/2594557
    
This contains a collection of playlists and the songs contained in them. I find all the unique songs and playlists and assign them ids. I then extract pairs (playlist_id, track_id) which can be used to create a collaborative filtering based recommender (See Create Model). We also save the artists,tracks and track ids to a SQL database (Table Tracks in Recommendations.db).

To make the computations more tracktable we restrict to the top_n (for now 10000) most common songs.

In [132]:
import pandas as pd
import json
import os
from os import path
import requests
import re
from zipfile import ZipFile


data_url = 'https://zenodo.org/record/2594557/files/spotify_playlists.zip?download=1'
csv_name = 'spotify_dataset.csv'

# To make the computations more tracktible we use only the n most common tracks
top_n = 10000
# Name to save the filtered data
processed_name = f'spotify_dataset_top{top_n}.csv'

In [133]:
# Download data if it is not already downloaded
if not path.isfile(csv_name):
    
    r = requests.get(data_url, allow_redirects=True)
    open('spotify_dataset.zip', 'wb').write(r.content)

    #unzip
    with ZipFile('spotify_dataset.zip', 'r') as zipObj:
        zipObj.extractall()
        
        
# Read into pandas dataframe
# Note the data has some lines with aditional commas, error_bad_lines = False allows us to skip these
df = pd.read_csv(csv_name,error_bad_lines = False).dropna()


In [134]:
#Function to clean standardize artist names
def normalize_artist(name):
    p = re.compile('&')
    name = p.sub(r'and', name)
    name = ' '.join(name.split())
    return name
 
#Function to clean standardize track names 
def normalize_track(name):
    p = re.compile('(\(|\[).{0,}(\)|\])')
    name = p.sub('', name)
    name = name.split('- ')[0]
    name = name.split('/')[0]
    name = ' '.join(name.split())
    return name


df.columns = ['user','artist','track','playlist']
df['artist'] = df['artist'].apply(normalize_artist)
df['track'] = df['track'].apply(normalize_track)

#Add fullname as unqiue identifier
df['full_name'] = df.artist + '-' + df.track


In [135]:
#Use ony the top n most common tracks
df_top = df[df['full_name'].isin(set(df.groupby('full_name').count()['user'].nlargest(top_n).index))]


In [136]:
class Indexer():
    '''
    Indexer contains a dictionary (indexed) of already seen songs and their indexes as well as a 
    counter max_index which gives the index of the last record, allowing us to assign a new number to unseen entries.
    '''
    
    def __init__(self):
        self.max_index = -1
        self.indexed = {}
    
    def get_index(self,x):
        '''
        Return the index of x if seen before, otherwise add it to indexed and return a new index.
        '''
        if x in self.indexed:
            return self.indexed[x]
        else:
            self.max_index += 1
            self.indexed[x] = self.max_index
            return self.max_index
        
    def __call__(self,x):
        '''
        Allow us to index via calling the object.
        '''
        return self.get_index(x)
    
    def save_index(self,path):
        '''
        Save the internal dictionary to a json
        '''
        with open(path, 'w') as f:
            json.dump(self.indexed, f)
            
    def __len__(self):
        return self.get_max()
    
    def get_max(self):
        return self.max_index+1
    

# Generate Index
track_indexer = Indexer()
playlist_indexer = Indexer()
df_top['playlist_id'] = df_top['playlist'].map(playlist_indexer)
df_top['track_id'] = df_top['full_name'].map(track_indexer)



<ipython-input-136-cbeb16fb100d>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top['playlist_id'] = df_top['playlist'].map(playlist_indexer)
<ipython-input-136-cbeb16fb100d>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top['track_id'] = df_top['full_name'].map(track_indexer)


In [137]:
#SAVE DATA

#Save name to track_id dictionary
track_indexer.save_index('name_to_id.json')

# Save top n to csv as (playlist_id,track_id) pairs
df_top[['playlist_id','track_id']].to_csv(processed_name,index=False)


In [138]:
{'num_playlists':len(playlist_indexer), 'num_tracks':len(track_indexer)}

{'num_playlists': 88878, 'num_tracks': 10000}

In [139]:
# Get unqiue tracks

df_tracks = df_top[['artist','track','track_id']].drop_duplicates()
df_tracks.set_index('track_id',inplace = True)
df_tracks

,artist,track
track_id,,
0,Elvis Costello,Alison
1,Crowded House,Don't Dream It's Over
2,Joshua Radin,I'd Rather Be With You
3,Paul McCartney,Live And Let Die
4,Joshua Radin,Only You
...,...,...
9995,Otto Klemperer/Sir Peter Pears/Dietrich Fische...,'St Matthew Passion' BWV244
9996,Fantomas,04
9997,Kerstin Gier,Silber


In [140]:
#Save unique tracks to a sql database
#https://docs.python.org/3/library/sqlite3.html

import sqlite3
conn = sqlite3.connect('recommendations.db')

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS Tracks")

df_tracks.to_sql('Tracks',conn)

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [161]:
import sqlite3
conn = sqlite3.connect('recommendations.db')

cursor = conn.cursor()

track_id = 0
artist = list(cursor.execute("SELECT Tracks.artist FROM Tracks WHERE track_id = ?", (track_id,)))[0][0]
print(artist)
result = cursor.execute("SELECT Tracks.artist, Tracks.track \
        FROM (SELECT recommended_id, distance FROM Recommendations WHERE track_id = ?) AS Recommended\
        INNER JOIN Tracks WHERE Tracks.track_id = Recommended.recommended_id AND Tracks.artist  <> ? \
        ORDER BY Recommended.distance LIMIT 5", (track_id, artist))

for r in result:
    print(r)

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

Elvis Costello
('Elvis Costello and The Attractions', 'Pump It Up')
('David Bowie', 'Young Americans')
("The B-52's", 'Rock Lobster')
('The Rolling Stones', 'Beast Of Burden')
('Smokey Robinson and The Miracles', 'The Tracks Of My Tears')
